# **Finding Lane Lines on the Road** 

## Goal
In this notebook I built a lane line detection pipeline for highway driving video image.  
Final output image is below.
<img src="examples/laneLines_thirdPass.jpg" width="480" alt="Combined Image" />

## Ideas for Lane Detection Pipeline
Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images  
`cv2.cvtColor()` to grayscale or change color  
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from sklearn.cluster import KMeans
%matplotlib inline

In [2]:
def mask(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    return cv2.bitwise_and(img, mask)

In [ ]:
image = mpimg.imread('test_images/solidWhiteRight.jpg')

# (1)Gray Scaled
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# (2)Gaussian Blur
blur = cv2.GaussianBlur(gray, (3, 3), 0)

# (3)Canny Edge Detection
canny = cv2.Canny(blur, 100, 120)

# (4)Masked unrelated area
masked = mask(canny, np.array([[(0, canny.shape[0]), (canny.shape[1]/2, 300), (canny.shape[1], canny.shape[0])]], dtype=np.int32))

lines = cv2.HoughLinesP(masked, 1, np.pi/180, 30, 5, 5)
S = (lines[:,:,3] - lines[:,:,1]) / (lines[:,:,2] - lines[:,:,0]) # slope
I = lines[:,:,1] - S * lines[:,:,0] # intercept

# (5)Hough Drawed
hough_drawed = np.zeros((canny.shape[0], canny.shape[1], 3), dtype=np.uint8)
for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(hough_drawed, (x1, y1), (x2, y2), [0,255,0], 2)
    
# (6)Hough Averaged
right = (S > 0.0).flatten()
left = (S < 0.0).flatten()

slope_r = S[right].mean()
intercept_r = I[right].mean()

slope_l = S[left].mean()
intercept_l = I[left].mean()

right_x = np.append(lines[:,:,0][right].flatten(), lines[:,:,2][right].flatten())
right_y = np.append(lines[:,:,1][right].flatten(), lines[:,:,3][right].flatten())
left_x = np.append(lines[:,:,0][left].flatten(), lines[:,:,2][left].flatten())
left_y = np.append(lines[:,:,1][left].flatten(), lines[:,:,3][left].flatten())

xr1 = np.min(right_x)
yr1 = np.min(right_y)
yr2 = image.shape[0]
xr2 = int((yr2-intercept_r) / slope_r)

yl1 = image.shape[0]
xl1 = int((yl1-intercept_l) / np.mean(slope_l))
xl2 = np.max(left_x)
yl2 = np.min(left_y)

result = np.zeros_like(hough_drawed)
cv2.line(result, (xr1, yr1), (xr2, yr2), [0, 255, 0], 4)
cv2.line(result, (xl1, yl1), (xl2, yl2), [0, 255, 0], 4)

print("Right:\t", slope_r, intercept_r)
print("Left: \t", slope_l, intercept_l)

In [ ]:
# show the way to detect lanes
plt.figure(figsize=(12, 12))
plt.subplot(3,2,1)
plt.title("(1)Gray Scaled")
plt.imshow(gray, cmap='gray')
plt.subplot(3,2,2)
plt.title("(2)Gaussian Blur")
plt.imshow(blur, cmap='gray')
plt.subplot(3,2,3)
plt.title("(3)Canny Edge")
plt.imshow(canny, cmap='gray')
plt.subplot(3,2,4)
plt.title("(4)Area Masked")
plt.imshow(masked, cmap='gray')
plt.subplot(3,2,5)
plt.title("(5)Hough Line")
plt.imshow(hough_drawed)
plt.subplot(3,2,6)
plt.title("(6)Averaged")
plt.imshow(result)

In [ ]:
# The program of this cell is not used. (I just want to keep my idea.)
# To cut off outliers of hough line I tried clustering.

#clusters = KMeans(n_clusters=2).fit_predict(np.abs(S))
#lane_line_cluster = 1 if np.mean(clusters) > 0.5 else 0
#not_noise = (clusters == lane_line_cluster)
#right = (S < 0.0).flatten()
#left = (S > 0.0).flatten()
#S_right = S[not_noise*right]
#S_left = S[not_noise*left]

## Build a Lane Finding Pipeline
To allpy above algorighm to video images I build a function as an interface.  
The program is almost same as above. The differende:
 - Step5 was cut off.
 - Add a process which overlays line drawed image to original image

In [3]:
def process_image(image):
    global global_slope_r, global_slope_l
    global global_intercept_r, global_intercept_l
    global xr1, yr1, xl2, yl2
    
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    canny = cv2.Canny(blur, 100, 120)
    masked = mask(canny, np.array([[(0, canny.shape[0]), (canny.shape[1]/2, 300), (canny.shape[1], canny.shape[0])]], dtype=np.int32))
    
    lines = cv2.HoughLinesP(masked, 1, np.pi/180, 30, 5, 5)
    S = (lines[:,:,3] - lines[:,:,1]) / (lines[:,:,2] - lines[:,:,0]) # slopes
    I = lines[:,:,1] - S * lines[:,:,0] # intercepts
    
    right = (S > 0.0).flatten()
    left = (S < 0.0).flatten()
    
    global_slope_r = global_slope_r * 0.8 + S[right].mean() * 0.2
    global_slope_l = global_slope_l * 0.8 + S[left].mean() * 0.2
    global_intercept_r = global_intercept_r * 0.8 + I[right].mean() * 0.2
    global_intercept_l = global_intercept_l * 0.8 + I[left].mean() * 0.2
    #print(global_slope_r, '\t', global_slope_l, '\t', global_intercept_r, '\t', global_intercept_l)
    
    right_x = np.append(lines[:,:,0][right].flatten(), lines[:,:,2][right].flatten())
    right_y = np.append(lines[:,:,1][right].flatten(), lines[:,:,3][right].flatten())
    left_x = np.append(lines[:,:,0][left].flatten(), lines[:,:,2][left].flatten())
    left_y = np.append(lines[:,:,1][left].flatten(), lines[:,:,3][left].flatten())
    
    xr1 = int(xr1 * 0.7 + np.min(right_x) * 0.3) if np.min(right_x) > xl2 + 5 else xr1
    yr1 = int(yr1 * 0.8 + np.min(right_y) * 0.2)
    yr2 = image.shape[0]
    xr2 = int((yr2-global_intercept_r) / global_slope_r)
    
    yl1 = image.shape[0]
    xl1 = int((yl1-global_intercept_l) / global_slope_l)
    xl2 = int(xl2 * 0.7 + np.max(left_x) * 0.3) if np.max(left_x) < xr1 - 5 else xl2
    yl2 = int(yl2 * 0.8 + np.min(left_y) * 0.2)
    #print(xr1, '\t', yr1, '\t', xl2, '\t', yl2)
    
    result = np.zeros_like(image)
    cv2.line(result, (xr1, yr1), (xr2, yr2), [255, 0, 0], 6)
    cv2.line(result, (xl1, yl1), (xl2, yl2), [255, 0, 0], 6)
    
    return cv2.addWeighted(image, 0.8, result, 1., 0.)

## Check Algorithm with Test Images

In [ ]:
import os
test_images = os.listdir("test_images/")

In [ ]:
test_image = mpimg.imread('test_images/{}'.format(test_images[3]))
plt.imshow(test_image)

In [ ]:
plt.imshow(process_image(test_image))

## Draw on Videos

In [4]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [5]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")

global_slope_r = 0.6379401735626508
global_slope_l = -0.5299387574062713
global_intercept_r = -0.7090874664852213
global_intercept_l = 646.7096210703668
xr1 = 502
yr1 = 318
xl2 = 467
yl2 = 316

white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!

%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|██████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:02<00:00, 107.85it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 2.26 s


In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [7]:
yellow_output = 'test_videos_output/solid_yellow_left.mp4'

global_slope_r = 0.6379401735626508
global_slope_l = -0.5299387574062713
global_intercept_r = -0.7090874664852213
global_intercept_l = 646.7096210703668
xr1 = 502
yr1 = 318
xl2 = 467
yl2 = 316

## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solid_yellow_left.mp4
[MoviePy] Writing video test_videos_output/solid_yellow_left.mp4


100%|██████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:05<00:00, 115.23it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solid_yellow_left.mp4 

Wall time: 6.13 s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

To clear this optional challenge I need to transform color space.